In [2]:
from IPython.display import display

In [1]:
import pandas as pd
import numpy as np
import os
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

from embedding import get_embedding, cos_sim
import breakdown

In [3]:
def prepare_dataframe(csv):
    df = pd.read_csv(csv)
    df["embedded"] = df["embedded"].map(lambda x: np.array(eval(x)))
    return df

In [4]:
def to_vectors(talker, comment):
    # 要素に分割
    sentences = eval(breakdown.get_list(talker, comment)) 

    # 各要素をembeddingしたリスト
    return [get_embedding(sentence) for sentence in sentences]

In [10]:
def ruizido(df, vector):
    """
    vector: 入力文の要素の一つをembedしたもの
    """
    for vector in 
    df[""] = df["embedded"].map(lambda x: cos_sim(x, vector))
    return df

In [5]:
df = prepare_dataframe("embedded.csv")
talker = "Agent[00]"
comment = "Agent[00]を占ったら黒だったよ。Agent[01]を占ったら白だったよ。"

for vector in to_vectors(talker, comment):
        print(ruizido(df, vector))

NameError: name 'ruizido' is not defined